In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import numpy as np
import matplotlib.pyplot as plt

In [ ]:

# Step 1: Data Preparation (Simulated)

# Example: Simulated Dataset (Replace with real student dataset)
num_samples = 2000  # Number of students
num_features = 20  # Example: 20 input features

# Randomly generated student features
X_train = np.random.rand(num_samples, num_features)
X_test = np.random.rand(500, num_features)

# Regression target: Final grades (0-100)
y_grade_train = np.random.rand(num_samples, 1) * 100
y_grade_test = np.random.rand(500, 1) * 100

# Classification target: At-risk students (0 or 1)
y_risk_train = np.random.randint(0, 2, (num_samples, 1))
y_risk_test = np.random.randint(0, 2, (500, 1))


In [ ]:

# Step 2: Model Definition (Shared Layers + Branching)

# Define Input Shape
input_shape = (num_features,)
inputs = Input(shape=input_shape)

# Shared Layers (Feature Extraction)
x = Dense(512, activation='relu')(inputs)
x = BatchNormalization()(x)
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)  # Regularization

# Task 1: Final Grade Prediction (Regression)
grade_branch = Dense(64, activation='relu')(x)
grade_branch = Dropout(0.2)(grade_branch)
grade_branch = Dense(32, activation='relu')(grade_branch)
grade_output = Dense(1, activation='linear', name='grade_output')(grade_branch)

# Task 2: At-Risk Classification (Binary Classification)
risk_branch = Dense(64, activation='relu')(x)
risk_branch = Dropout(0.2)(risk_branch)
risk_branch = Dense(32, activation='relu')(risk_branch)
risk_output = Dense(1, activation='sigmoid', name='risk_output')(risk_branch)

# Define the Multi-Task Model
model = Model(inputs=inputs, outputs=[grade_output, risk_output])


In [ ]:

# Step 3: Compile Model with Multi-Task Loss


# Custom loss weights: Adjust task importance (1.0 means equal priority)
loss_weights = {'grade_output': 1.0, 'risk_output': 0.5}

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss={
        'grade_output': 'mse',  # Mean Squared Error for Regression
        'risk_output': 'binary_crossentropy'  # Binary Cross-Entropy for Classification
    },
    loss_weights=loss_weights,
    metrics={
        'grade_output': 'mae',  # Mean Absolute Error
        'risk_output': 'accuracy'  # Classification Accuracy
    }
)

# Model Summary
model.summary()

In [ ]:

# Step 4: Train the Model

# Callbacks for better training
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5)

history = model.fit(
    X_train,
    {'grade_output': y_grade_train, 'risk_output': y_risk_train},
    validation_data=(X_test, {'grade_output': y_grade_test, 'risk_output': y_risk_test}),
    epochs=100,
    batch_size=32,
    callbacks=[early_stopping, reduce_lr]
)


In [ ]:

# Step 5: Evaluate the Model

results = model.evaluate(X_test, {'grade_output': y_grade_test, 'risk_output': y_risk_test})
print("\nTest Results:", results)


In [ ]:

# Step 6: Make Predictions (Custom Dummy Data)


# Example: Custom test student data (Replace with real values)
sample_students = np.array([
    [0.8, 0.6, 0.5, 0.7, 0.9, 0.3, 0.4, 0.2, 0.8, 0.9, 0.1, 0.7, 0.6, 0.3, 0.5, 0.8, 0.9, 0.6, 0.4, 0.7],  # Student 1
    [0.4, 0.5, 0.2, 0.6, 0.3, 0.8, 0.9, 0.7, 0.1, 0.4, 0.6, 0.3, 0.5, 0.7, 0.9, 0.1, 0.2, 0.8, 0.9, 0.5]   # Student 2
])

# Make predictions
predictions = model.predict(sample_students)

# Print Results
for i in range(len(sample_students)):
    print(f"\nStudent {i+1}:")
    print(f"   Predicted Final Grade: {predictions[0][i][0]:.2f}")
    print(f"   At-Risk Probability: {predictions[1][i][0]:.2f} (Threshold 0.5)")


In [ ]:

# Step 7: Plot Training History


def plot_training(history):
    # Plot loss
    plt.figure(figsize=(12, 5))
    
    # Loss Plot
    plt.subplot(1, 2, 1)
    plt.plot(history.history['grade_output_loss'], label='Grade Loss')
    plt.plot(history.history['risk_output_loss'], label='Risk Loss')
    plt.plot(history.history['val_grade_output_loss'], label='Val Grade Loss', linestyle='dashed')
    plt.plot(history.history['val_risk_output_loss'], label='Val Risk Loss', linestyle='dashed')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Training Loss')
    plt.legend()

    # Accuracy Plot
    plt.subplot(1, 2, 2)
    plt.plot(history.history['risk_output_accuracy'], label='Risk Accuracy')
    plt.plot(history.history['val_risk_output_accuracy'], label='Val Risk Accuracy', linestyle='dashed')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.title('Classification Accuracy')
    plt.legend()

    plt.show()

plot_training(history)